# [과제 3] 로지스틱 회귀분석
### - sklearn 패키지를 사용해 로지스틱 회귀분석을 진행해주세요.
### - 성능지표를 계산하고 이에 대해 해석해주세요.
### - 성능 개선을 시도해주세요. (어떠한 성능지표를 기준으로 개선을 시도했는지, 그 이유도 함께 적어주세요.)
### - 주석으로 설명 및 근거 자세하게 달아주시면 감사하겠습니다. :)

In [173]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data

출처 : https://www.kaggle.com/mlg-ulb/creditcardfraud


* V1 ~ V28 : 비식별화 된 개인정보
* **Class** : Target 변수  
  - 1 : fraudulent transactions (사기)
  - 0 : otherwise

In [174]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings(action='ignore')

In [175]:
data = pd.read_csv("/content/drive/MyDrive/21기 투빅스/assignment3_creditcard.csv")

In [176]:
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Class
0,-1.848212,2.384900,0.379573,1.048381,-0.845070,2.537837,-4.542983,-10.201458,-1.504967,-2.234167,...,2.585817,-5.291690,0.859364,0.423231,-0.506985,1.020052,-0.627751,-0.017753,0.280982,0
1,2.071805,-0.477943,-1.444444,-0.548657,0.010036,-0.582242,-0.042878,-0.247160,1.171923,-0.342382,...,-0.077306,0.042858,0.390125,0.041569,0.598427,0.098803,0.979686,-0.093244,-0.065615,0
2,-2.985294,-2.747472,1.194068,-0.003036,-1.151041,-0.263559,0.553500,0.635600,0.438545,-1.806488,...,1.345776,0.373760,-0.385777,1.197596,0.407229,0.008013,0.762362,-0.299024,-0.303929,0
3,-1.479452,1.542874,0.290895,0.838142,-0.529290,-0.717661,0.484516,0.545092,-0.780767,0.324804,...,0.038397,0.116771,0.405560,-0.116453,0.541275,-0.216665,-0.415578,0.027126,-0.150347,0
4,-0.281976,-0.309699,-2.162299,-0.851514,0.106167,-1.483888,1.930994,-0.843049,-1.249272,1.079608,...,-0.875516,-0.004199,1.015108,-0.026748,0.077115,-1.468822,0.751700,0.496732,0.331001,0


In [177]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28678 entries, 0 to 28677
Data columns (total 29 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      28678 non-null  float64
 1   V2      28678 non-null  float64
 2   V3      28678 non-null  float64
 3   V4      28678 non-null  float64
 4   V5      28678 non-null  float64
 5   V6      28678 non-null  float64
 6   V7      28678 non-null  float64
 7   V8      28678 non-null  float64
 8   V9      28678 non-null  float64
 9   V10     28678 non-null  float64
 10  V11     28678 non-null  float64
 11  V12     28678 non-null  float64
 12  V13     28678 non-null  float64
 13  V14     28678 non-null  float64
 14  V15     28678 non-null  float64
 15  V16     28678 non-null  float64
 16  V17     28678 non-null  float64
 17  V18     28678 non-null  float64
 18  V19     28678 non-null  float64
 19  V20     28678 non-null  float64
 20  V21     28678 non-null  float64
 21  V22     28678 non-null  float64
 22

In [178]:
data.isnull().sum()

V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
V11      0
V12      0
V13      0
V14      0
V15      0
V16      0
V17      0
V18      0
V19      0
V20      0
V21      0
V22      0
V23      0
V24      0
V25      0
V26      0
V27      0
V28      0
Class    0
dtype: int64

In [179]:
data['Class'].value_counts()

0    28432
1      246
Name: Class, dtype: int64

➡️성능 개선 때 오버샘플링 진행

In [180]:
X = data.drop('Class', axis=1)
y = data['Class']### 정규화

### 정규화

In [181]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(X)

array([[-0.90218019,  1.44246214,  0.24207645, ..., -1.29584288,
        -0.04709654,  0.9417547 ],
       [ 1.02744235, -0.32892601, -0.7953942 , ...,  2.02637926,
        -0.23721911, -0.22629752],
       [-1.46190736, -1.73320058,  0.7053479 , ...,  1.57721728,
        -0.7554671 , -1.02943014],
       ...,
       [ 1.02786291,  0.0843943 , -0.9747001 , ...,  0.36917223,
        -0.15087861, -0.1030219 ],
       [-0.33236542,  0.79733419,  0.8782926 , ...,  0.21096583,
        -2.140772  , -1.05999649],
       [ 1.09831537, -0.52987507, -0.90637085, ..., -0.61547609,
        -0.13776702, -0.2565925 ]])

### 모델링

In [182]:
# train, test set 분리
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state = 0)

In [183]:
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)


(22942, 28) (5736, 28) (22942,) (5736,)


In [184]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train, y_train)

LogisticRegression()

In [185]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = lr.predict(x_test)

### 평가지표

- Accuracy (정확도)
    - 정확도는 종합적인 지표이기 때문에 세부적인 지표의 값을 가리고 있다
    - 특히나 **편향된 데이터**에 대해서는 선호되지 않는다
    
- Precision (정밀도)
    - positive 라고 예측한 인스턴스 중에서 실제로 positive 인 인스턴스의 비율
    - Precision = TP / FP + TP
- Recall (재현율)
     - actual positive 중에서 얼마나 잘 정확하게 예측했는지에 대한 비율
     - Recall = TP / FN + TP
- F1-score
    - 정밀도와 재현율의 조화 평균
    - 편향된 데이터셋에서 모델 성능을 평가하는 데 더 유용하다
    
    $F_1 score = {2 (precision \times recall) \over (precision + recall)}$
     
     
- **Confusion matrix**

|  | Negative | Positive |
| --- | --- | --- |
| actual Negative | TN | FP |
| actual Positive | FN | TP |

✅ 사기 탐지를 목적으로 문제를 해결할 때 정밀도와 재현율 중 어느 것을 높여야 할까?

- 고객들의 불쾌감을 덜어주려면 **high precision**
- 범죄를 더 많이 확실하게 예방하기 위함이라면 **high recall**

➡️  실제로 Positive인 얘들을 Negative로 예측하는 것이 위험이 더 크기 때문에, recall 성능을 높이는 데 초점

- 단, 두 지표는 Trade-off 관계이므로, Threshold (임계값)을 조절하여 진행하면 두 지표 사이의 균형을 조절할 수 있다
- Threshold를 낮추면, Positive로 예측하는 값이 더 많아지게 되니 Precision은 감소하고, Recall은 증가

In [186]:
print("Accuracy:")
print(round(accuracy_score(y_test, y_pred),5))

Accuracy:
0.99808


In [187]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Confusion Matrix:
[[5690    4]
 [   7   35]]


In [188]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5694
           1       0.90      0.83      0.86        42

    accuracy                           1.00      5736
   macro avg       0.95      0.92      0.93      5736
weighted avg       1.00      1.00      1.00      5736



### 성능 지표 개선

1) 오버 샘플링 : 데이터 불균형 문제 해소

In [189]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

X_train_over, y_train_over = smote.fit_resample(x_train, y_train)
# 독립변수 비율 확인
y.value_counts()

0    28432
1      246
Name: Class, dtype: int64

In [190]:
# 독립변수 비율 확인
y_train_over.value_counts()

0    22738
1    22738
Name: Class, dtype: int64

In [191]:
lr2 = LogisticRegression()
lr2.fit(X_train_over, y_train_over)

y_pred2 = lr2.predict(x_test)

In [192]:
print("Accuracy:")
print(round(accuracy_score(y_test, y_pred2),4))

Accuracy:
0.9719


In [193]:
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred2))

Confusion Matrix:
[[5534  160]
 [   1   41]]


In [194]:
print("Classification Report:")
print(classification_report(y_test, y_pred2))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      5694
           1       0.20      0.98      0.34        42

    accuracy                           0.97      5736
   macro avg       0.60      0.97      0.66      5736
weighted avg       0.99      0.97      0.98      5736



- recall 값은 향상되었으나, f1-score 값과 precision 값이 매우 낮게 나옴

2) 그리드 서치 : 하이퍼파라미터의 최적의 조합을 찾아 모델의 성능 향상
- 정밀도와 재현율의 균형을 맞추기 위해 f1 값을 성능 지표로 지정


In [216]:
from sklearn.model_selection import GridSearchCV

params = {'C' : [0.005, 0.01, 0,1 ,1, 10], 'penalty': ['l1', 'l2']}

grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid =params, cv=5, scoring='f1')
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.005, 0.01, 0, 1, 1, 10],
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [217]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best f1 score: {grid_search.best_score_}")

Best parameters: {'C': 1, 'penalty': 'l2'}
Best f1 score: 0.8250562223164964


In [218]:
# 최적의 하이퍼파라미터로 예측하기
best_params = grid_search.best_estimator_
y_pred3 = best_params.predict(x_test)

In [219]:
print("Classification Report:")
print(classification_report(y_test, y_pred3))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5694
           1       0.90      0.83      0.86        42

    accuracy                           1.00      5736
   macro avg       0.95      0.92      0.93      5736
weighted avg       1.00      1.00      1.00      5736



In [215]:
# 오버샘플링 + 그리드서치

grid_search = GridSearchCV(estimator = LogisticRegression(), param_grid =params, cv=5, scoring='f1')
grid_search.fit(X_train_over, y_train_over)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.005, 0.01, 0, 1, 1, 10],
                         'penalty': ['l1', 'l2']},
             scoring='f1')

In [206]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best f1 score: {grid_search.best_score_}")

Best parameters: {'C': 10, 'penalty': 'l2'}
Best f1 score: 0.9479460618062051


In [204]:
# 최적의 하이퍼파라미터로 예측하기
best_params = grid_search.best_estimator_
y_pred4 = best_params.predict(x_test)

In [205]:
print("Classification Report:")
print(classification_report(y_test, y_pred4))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      5694
           1       0.20      0.98      0.34        42

    accuracy                           0.97      5736
   macro avg       0.60      0.97      0.66      5736
weighted avg       0.99      0.97      0.98      5736



- 그리드 서치를 진행한 2가지 결과 모두 이전 결과와 동일하게 나타남
- 성능 개선을 위한 다른 방법을 고안하는 것이 필요해보임